<div style="text-align: right"><strong>Capstone #3:</strong> <span style="color:darkred">Supervised Learning</span> </div>

<a id="top"></a>

#### <span style="color:darkred">__Part 1: Data Exploration__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-exploration.ipynb </span><br><br><span style="color:darkred">__Part 2: Models__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-models.ipynb </span>

----

# <span style="color:darkred">Part 2: </span><span style="color:darkblue">L1 Prediction from ELL Writing Samples</span>

__Author:__ Ryan Harper 

----

<a href='#ov'>Overview</a><br>
<a href='#exp'>Experiment</a><br>
<a href='#sec1'>1. Models:</a><br>
><a href='#seca'>A. LR - Ordinary Least Squares</a><br>
<a href='#secb'>B. LR - Logistic Regression</a> <a href='#secb1'> (Lasso)</a> <a href='#secb2'> (Ridge)</a><br>
<a href='#secc'>C. NN - K Nearest Neighbors</a><br>
<a href='#secd'>D. NN - Naive Bayes</a><br>
<a href='#sece'>E. NN - Decision Tree</a><br>
<a href='#secf'>F. Ensemble - Random Forest</a><br>

<a href='#sec2'>2. Model Comparison</a><br>

<a id="ov"></a>

# <span style="color:darkblue">Overview</span>  <a href='#top'>(top)</a>

__Data Source:__
> http://lang-8.com/ [scraped with Beautiful Soup]

![alt text](../data/language/lang8.png "Title")

__Summary:__
> In my previous profession, I have been teaching English to a diverse range of students of all ages, language background, and country origin. During my professional development, I started to observe that different students with different L1s (1st Language) tended to display different patterns of communication that appeared to have some connection to either education in their country of origin or a connection to the linguistic structure of their first language. Different ELL (English Language Learners) needed to focus on different aspects of the English language depending on their background. The purpose of this project is to use a large number of blog posts from a language practicing website and explore whether or not the L1 has any significant impact on the blog writing style of the English learner.<br><br>Part 1: Explore the data to find any noteworthy trends in linguistic structure: <ol><li> vocabulary (word freq, collocations, and cognates) <li>syntax (sentence structure)<li>grammar (i.e. grammar complexity of sentences) <li>errors (types of errors) <li> parts of speech (NLTK Abbreviations: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/)</ol><br>Part 2: Use linguistic trends to determine whether or not a learner's first language can be predicted.

__Variables:__
>__id:__ _User ID_<br>
__time:__ _Time the blog post was scraped (in order of user posted time)_ <br>
__title:__ _Title of the blog post_<br>
__content:__ _The blog post_<br>
__language:__ _User's self-reported first language_

<a id="exp"></a>

# <span style="color:darkblue">Experiment</span> <a href='#top'>(top)</a>

__Hypothesis:__ 
> L1 (first language) experience and academic environment influences ELLs' (English Language Learners') writing style. The L1 of ELLs can be predicted by looking at English blog posts and identifying patterns unique to their L1.

__Observations:__
><li> --<li>--<li>--

__Method:__
> Using multiple different models. The aim of this project is to explore how different models can handle the data (target and features) and to see what information can be gained from using multiple different models. Ultimately, the goal is to determine which models are appropriate for a binary (discrete) target with features that are both qualitative (discrete) and quantitative (ranked/continuous).

<a id="sec1"></a>

# <span style="color:darkblue">1. Models:</span>  <a href='#top'>(top)</a>

In [1]:
# iPython/Jupyter Notebook
import time
from pprint import pprint
import warnings
from IPython.display import Image

import time

# Data processing
import pandas as pd
import plotly as plo
import seaborn as sns
from scipy import stats
from collections import Counter
import numpy as np
import itertools

# NLP
from nltk.corpus import stopwords as sw
from nltk.util import ngrams
from nltk.corpus import brown
import nltk
import re
from nltk.tokenize import RegexpTokenizer
import difflib

# Stats
from sklearn.metrics import classification_report, roc_curve,roc_auc_score,accuracy_score
from sklearn import metrics

# Preparing Models
from sklearn.model_selection import train_test_split

# Decomposition
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

# Models
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB

# Ensemble
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

#Visualization
from IPython.display import Image
import pydotplus
import graphviz

# import altair as alt

__Import Features + Target__

In [2]:
features = pd.read_csv('blogfeatures.csv').sample(frac=1.0)
del features['Unnamed: 0']
del features['id']
del features['content']
del features['pos']
del features['pos2']
del features['pos3']
del features['tokens']

lang = list(features.language.unique())

In [3]:
features.language.value_counts()

Japanese               9536
Traditional Chinese    3407
Korean                 1122
English                  65
Name: language, dtype: int64

__Declare X,y Variables__

In [4]:
y = features['language'].values.reshape(-1, 1).ravel()
X = features[features.columns[~features.columns.str.contains('language')]]
X.head()

print(np.shape(y))
print(np.shape(X))

(14130,)
(14130, 14835)


__SVD Truncate: To find most important features__

In [5]:
svd = TruncatedSVD(n_components=20, n_iter=7, random_state=42)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=20, n_iter=7,
       random_state=42, tol=0.0)

In [6]:
best_features = [X.columns[i] for i in svd.components_[0].argsort()[::-1]]

__Reduce Features (Unused): 14,665 to n_components __

__Split Data to Train/Test__

_Using TrainTestSplit (Unused)_

_Even Distribution Sampling_

In [7]:
# evenly_distributed_test = [60 japanese,60 english, 60 chinese, 60 korean]
rs=33
japset = features[features['language'] == 'Japanese'].sample(n=30, random_state=rs)
korset = features[features['language'] == 'Korean'].sample(n=30, random_state=rs)
chiset = features[features['language'] == 'Traditional Chinese'].sample(n=30, random_state=rs)
engset = features[features['language'] == 'English'].sample(n=30, random_state=rs)
testset = [japset,korset,chiset,engset]

In [8]:
test_data = pd.concat(testset)
y_test = test_data['language'].values.reshape(-1, 1).ravel()
X_test = test_data[best_features]

In [9]:
train_data = features[~features.isin(test_data)].dropna()
y_train = train_data['language'].values.reshape(-1, 1).ravel()
X_train = train_data[best_features]
# X_train = svd.transform(train_data[train_data.columns[~train_data.columns.str.contains('language')]])

<a id="seca"></a>

__Create Function for Comparing Models__

In [10]:
cols = ['name','time','total','precision','recall','f1']

model_set = pd.DataFrame(columns=cols)
models_stored = []
pattern = "%.2f"

In [11]:
def run_model(model,name):
    global model_set
    m = model
    m.fit(X_train, y_train)
    start = time.time()

    total_score = m.score(X_test,y_test)
    pscore = [pattern % i for i in list(metrics.precision_score(y_test, m.predict(X_test),labels=lang,average=None))]
    rscore = [pattern % i for i in list(metrics.recall_score(y_test, m.predict(X_test),labels=lang,average=None))]
    fscore = [pattern % i for i in list(metrics.f1_score(y_test, m.predict(X_test),labels=lang,average=None))]
    end = time.time()
    t= pattern % (end - start)

    r = dict(zip(cols,[name,t,total_score,pscore,rscore,fscore]))
    print('Check for Overfitting: {}\n'.format(m.score(X_train,y_train)))
    print('Test Score is: {}\n'.format(total_score))
    print(classification_report(y_test, m.predict(X_test)))
    
    model_set = model_set.append(r,ignore_index=True)
    return r,m

<a id="seca"></a>

### <span style="color:darkred">A. LR - Logistic Regression</span>  <a href='#top'>(top)</a>

> Target is binary so logistic regression will operate on probabilities

In [ ]:
%%time
lreg_data,lreg = run_model(linear_model.LogisticRegression(),'Logistic Regression')

In [ ]:
len(lreg.predict_proba(X_train)[0])

In [ ]:
pd.crosstab(y_test,lreg.predict(X_test))

In [ ]:
# print(np.std(X)*lreg.coef_)

<a id="secb1"></a>

<a id="sece"></a>

### <span style="color:darkred">E. K Nearest Neighbors</span>  <a href='#top'>(top)</a>

> Can handle discrete values for target <br>Quantitative values are limited (not continuous) and might be problematic for nearest neighbors

In [ ]:
%%time
neighbors_data,neighbors = run_model(KNeighborsClassifier(n_neighbors=5),'K Nearest Neighbor')

<a id="secf"></a>

### <span style="color:darkred">F. Naive Bayes - Bernoulli</span>  <a href='#top'>(top)</a>

> Should be best for boolean classification but not for multiple discrete values (multinomial should be better)

In [ ]:
%%time
bnb_data,bnb = run_model(GaussianNB(),'Naive Bayes - Bernoulli')

<a id="secg"></a>

### <span style="color:darkred">G. Decision Tree</span>  <a href='#top'>(top)</a>

> Visualizes most important features by hierarchy <br>Longer processing time

In [ ]:
%%time
dt_data,dt = run_model(tree.DecisionTreeClassifier(criterion='entropy',max_depth=5),'Decision Tree')

In [ ]:
# Render tree.
dot_data = tree.export_graphviz(
    dt, 
    out_file=None,
    feature_names=X_train.columns,
    label= 'root',
    proportion=False,
    rounded=True,
    class_names=lang,
    filled=True
)

graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

graph.write_png('decision_tree.png')

In [ ]:
dimportance = list(zip(best_features,dt.feature_importances_))
dimportance = dict(dimportance)
a1_sorted_keys = sorted(dimportance, key=dimportance.get, reverse=True)
p = []
for r in a1_sorted_keys:
    if dimportance[r] != 0:
        p.append(r)
#         print(r, dimportance[r])
        
print(p)

_Good visualization of important features and presentation of entropy weighting_

<a id="sech"></a>

### <span style="color:darkred">H. Random Forest</span>  <a href='#top'>(top)</a>

> Runs decision tree multiple times for best output <br>Longest processing time

In [ ]:
%%time
rf_data,rf = run_model(ensemble.RandomForestClassifier(n_estimators=8,criterion='entropy',max_features=len(X_train.columns),max_depth=5),'Random Forest')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50)
clf = clf.fit(X_train, Y_train)

featureImportance = clf.feature_importances_
for i in xrange(len(featureImportance)):
    print featureImportance[i]

<a id="sec2"></a>

# <span style="color:darkblue">2. Model Comparison</span>  <a href='#top'>(top)</a>

In [ ]:
model_set.columns = ['name','time','total','prec: | JA | CH | KO | EN |','rec: | JA | CH | KO | EN |','f1: | JA | CH | KO | EN |']
model_set

-----

Regularization